In [51]:
import surprise
from surprise import SVD
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd

In [52]:
ratings = pd.read_csv('data/movie_rating.csv')
ratings.head()

,critic,title,rating
0,Jack,Lady,3.0
1,Jack,Snakes,4.0
2,Jack,You Me,3.5
3,Jack,Superman,5.0
4,Jack,The Night,3.0


In [55]:
movies2 = set(ratings['title'])
print(movies2)
print(len(movies2))

{'The Night', 'Just My', 'Superman', 'You Me', 'Snakes', 'Lady'}
6


In [56]:
reader = Reader(rating_scale=(0.5, 5.0))

In [63]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='data/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [91]:
algo = SVD(n_factors=20, n_epochs= 30,  random_state=1)
algo.fit(trainset) 
#predictions = algo.test( testset )
#accuracy.rmse(predictions)

In [92]:
from surprise.model_selection import cross_validate

In [93]:
movies = pd.read_csv('data/movie_rating.csv')
set(movies['title'])

{'Just My', 'Lady', 'Snakes', 'Superman', 'The Night', 'You Me'}

In [94]:
uid = str('Toby')
iid = str('Lady')

pred = algo.predict(uid, iid, verbose=True)

user: Toby       item: Lady       r_ui = None   est = 3.50   {'was_impossible': False}


In [95]:
def get_unseen_surprise2(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['critic']== userId]['title'].tolist()
    print('본 영화수 >> ', len(seen_movies))

    # 모든 영화들의 movieId를 리스트로 생성. 
    print('전체 영화수 >> ', len(movies))

    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), '전체 영화수:',len(movies))    
    return unseen_movies

unseen_movies2 = get_unseen_surprise2(ratings, movies2, 'Toby')

본 영화수 >>  3
전체 영화수 >>  6
평점 매긴 영화수: 3 추천대상 영화수: 3 전체 영화수: 6


In [96]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=2):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(one):
        return one.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [pred.iid for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_preds = list(zip(top_movie_ids, top_movie_rating))
    
    return top_movie_preds

In [97]:
top_predictions = recomm_movie_by_surprise(algo, 'Toby', unseen_movies2)
for index, top_movie in enumerate(top_predictions):
        print(index+1,'. 추천 영화 :',top_movie[0], "\t, 평점 :", top_movie[1])


1 . 추천 영화 : The Night 	, 평점 : 3.501556983616962
2 . 추천 영화 : Just My 	, 평점 : 3.501556983616962
